In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ids=[
"3578243.ECG2177606",
"06233283.ECG2442913",
"7195417.ECG3710114",
"11230265.ECG2957930",
"11800116.ECG3975923",
"11959129.ECG2437308",
"14986364.ECG3494515",
"19362211.ECG2112908",
"19994946.ECG2155511",
"24469849.ECG3704104",
"26283283.ECG2155507",
"26774436.ECG3561307",
"26973046.ECG3710114",
"27652160.ECG3852014",
"29113631.ECG3270523",
"30798324.ECG3716528",
"33455004.ECG2651910",
"35915481.ECG2651902",
"35923967.ECG3377508",
"36000632.ECG3775816",
"36052809.ECG8208",
"36187209.ECG2437304",
"36210830.ECG2957930",
"36227926.ECG2112908",
"36228849.ECG3386501",
"36265880.ECG2031307",
"36267319.ECG3851518",
"0428191911.ECG3561327"]

In [3]:
num=1
data=pd.DataFrame()
for i in ids:
    RRI=pd.read_table("/Users/erikafox/Desktop/Heart-Rate-Variability-Repo/01_data/1st topic/"+i+"AnesEvents-6+10mins_RRI.txt",sep=",",names=["time","signal"])
    BIS=pd.read_table("/Users/erikafox/Desktop/Heart-Rate-Variability-Repo/01_data/1st topic/"+i+"AnesEvents-6+10mins_BIS.txt",sep=",",names=["time","signal"])
    temp=pd.DataFrame( {'RRI_patient_'+str(num):RRI["signal"],'BIS_patient_'+str(num):BIS["signal"] } )
    print(temp)
    num+=1
    data=pd.concat([data, temp],axis=1)

      RRI_patient_1  BIS_patient_1
0        947.496594      38.933975
1        946.049856      38.744022
2        950.959205      38.554069
3        959.596392      38.364116
4        969.281723      38.174163
...             ...            ...
3836     671.128431      26.179041
3837     669.305195      26.219031
3838     667.937178      26.259021
3839     667.119144      26.299011
3840     666.889703      26.334126

[3841 rows x 2 columns]
      RRI_patient_2  BIS_patient_2
0        982.034641      40.631112
1        977.966327      40.726064
2        975.914210      40.821017
3        977.147376      40.915969
4        981.219561      41.010921
...             ...            ...
3836    1140.766647      28.717459
3837    1141.498737      28.712461
3838    1142.230828      28.707464
3839    1142.962918      28.702466
3840    1143.695008      28.700000

[3841 rows x 2 columns]
      RRI_patient_3  BIS_patient_3
0       1055.712962      45.324064
1       1053.145867      45.194152
2    

In [4]:
def getRes(yhat,test):
    res=pd.DataFrame()
    for c in data.columns:
        s="pred_"+str(c)
        t=pd.DataFrame({s:yhat[c],c:test[c]})
        res=pd.concat([res, t],axis=1)
    return res

In [5]:
from statsmodels.tsa.statespace.varmax import VARMAX
from random import random

def show_graph(df1,df2,title):
    data = pd.concat([df1, df2])
    data.reset_index(inplace=True, drop=True)
    for col in data.columns:
        if col.lower().startswith('pred'):
            data[col].plot(label=col,linestyle="dotted")
        else:
            data[col].plot(label=col)
    plt.title(title)
    plt.legend()
    plt.show()

def VARMA_model(train,test):
    # fit model
    model = VARMAX(train, order=(1, 2))
    model_fit = model.fit(disp=False)
    # make prediction
    yhat = model_fit.forecast(steps=len(test))
    res=getRes(yhat,test)
    return res

In [6]:

def getRes(yhat,test):
    res=pd.DataFrame()
    for c in data.columns:
        s="pred_"+str(c)
        t=pd.DataFrame({s:yhat[c],c:test[c]})
        res=pd.concat([res, t],axis=1)
    return res


In [7]:
def getTrainTest(df,cutoff):
    df_train = df[df.index<cutoff]
    df_test =  df[df.index>=cutoff]
    return df_train,df_test


In [8]:
df_train,df_test=getTrainTest(data,3500)
df_ret = VARMA_model(df_train, df_test)


/opt/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/opt/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/varmax.py:374: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


KeyboardInterrupt: 

In [ ]:
show_graph(df_train, df_ret)